In [16]:
!pip install pyarrow
!pip install torch-geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


# Helper function for visualization.

In [ ]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt


def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

In [19]:
import pyarrow.feather as feather
table = feather.read_table('dataset/data_andre.feather', memory_map=True)

In [ ]:
import pyarrow as pa
import pyarrow.compute as pc
target_col  = 'value'
series_keys = ['store_id', 'item_id']
promo_pairs = ['FRPG','GAS','BOGO','DISC','CIRC','CIRE','CLCP','LFPE']
dynamic_cols = [f'promo_type_{p}' for p in promo_pairs] + \
               [f'promo_value_{p}' for p in promo_pairs]

# If you want simple time features without pandas:
# day of week from timestamp (Mon=0)
dow = pc.day_of_week(table['date']) - 1
table = table.append_column('dow', dow)  # int32
dynamic_cols += ['dow']

# Static categoricals you might embed later (we'll keep as-is or dictionary-encode)
static_cat = ['cat_label','sdep_label','dep_label','dmn_label']
